In [ ]:
import asyncio
from jupyasyncclient import JupyAsyncKernelClient,JupyAsyncKernelManager

## Quick start

In [ ]:
import os, socket, subprocess, sys, time, urllib.request, pytest

In [ ]:
def free_port() -> int:
    s = socket.socket()
    s.bind(("127.0.0.1", 0))
    port = s.getsockname()[1]
    s.close()
    return port

In [ ]:
free_port()

57867

In [ ]:
def jp_server(root='.', host='127.0.0.1', port=None, timeout=30):
    if not port: port = free_port()
    base_url = f"http://{host}:{port}"
    cmd = [ sys.executable,
        "-m", "jupyter_server", "--no-browser",
        f"--ServerApp.port={port}", "--ServerApp.port_retries=0",
        "--ServerApp.token=", "--ServerApp.password=",
        "--ServerApp.disable_check_xsrf=True",
        "--ServerApp.allow_root=True", f"--ServerApp.root_dir={root}",
        "--ServerApp.open_browser=False", "--ServerApp.log_level=50",
    ]
    env = dict(os.environ)
    env.setdefault("JUPYTER_PLATFORM_DIRS", "1")

    proc = subprocess.Popen(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, env=env)
    deadline = time.time()+timeout
    while time.time() < deadline:
        try:
            urllib.request.urlopen(base_url + "/api/kernelspecs", timeout=1).read()
            break
        except Exception: time.sleep(0.2)
    else: raise RuntimeError("Jupyter Server did not start")
    return proc,base_url

In [ ]:
def stop(proc):
    proc.terminate()
    try: proc.wait(timeout=10)
    except Exception: proc.kill()

In [ ]:
proc,base_url = jp_server()

In [ ]:
# host,port = '127.0.0.1',9999
# url = f"http://{host}:{port}"
token = ''

In [ ]:
km = JupyAsyncKernelManager(base_url, token=token)
await km.start_kernel()
kc = km.client()
kc.start_channels()
await kc.wait_for_ready(timeout=30)

Or:
```py
km, kc = await start_new_server_kernel(url, token=token)
```

In [ ]:
rep = await kc.execute_interactive("print('hello')", timeout=30)
print(rep["content"]["status"])

hello
ok


In [ ]:
await kc.aclose()
await km.shutdown_kernel(now=True)
await km.aclose()

In [ ]:
stop(proc)